# Exercise set 4 : Convex optimisation problems

In [9]:
import numpy as np
import cvxpy as cp

## Ex 1 : minimise fresh wood use
Demand : $d_j$, percentage in fresh paper : $m_j$, recycling efficiency : $t_j$, amount of recycling paper available $s_j$, $j=5$ denotes the different types of paper which can be used.

In [28]:
# Parameters and variables
fresh_proportion = cp.Variable(5)
recycling_repartition = cp.Variable((5,5))
recycled_proportion = 1 - fresh_proportion
demand = np.array([3475,1223,2260,2700,2950])
percentage_fresh = np.array([0,.47,.5,.4,.3])
old_paper = np.array([2000,1600,1000,990,2800])
recycling_rate = np.array([.85,.9,.85,.85,.9])
old_available = np.multiply(old_paper,recycling_rate)
recycling_used_for = np.array([[1,1,0,0,0],[1,1,1,1,0],[0,1,1,1,1],[0,1,1,0,1],[0,0,0,0,1]])

# Constraints
objective = cp.sum(fresh_proportion)
constraints = [fresh_proportion >= percentage_fresh , 
               cp.multiply((1-fresh_proportion),demand) == cp.sum(cp.multiply(recycling_repartition,recycling_used_for.T),axis=1) ,
               recycling_repartition.T@np.ones(5) <= np.multiply(old_paper,recycling_rate), 
               recycling_repartition >= np.zeros((5,5))]

# Solving
prob = cp.Problem(cp.Minimize(objective), constraints)
prob.solve(solver=cp.MOSEK, verbose=True)

# Display solution 
print('The fresh paper quantities for each type of paper are : \n',fresh_proportion.value,' >= ',percentage_fresh)
print('Recycling paper repartition : \n',recycling_repartition.value)
print('Paper used from each source : \n',np.sum(recycling_repartition.value,axis=0),' <= ',old_available)
print('Total quantity of fresh wood : ',fresh_proportion.value@demand)

                                     CVXPY                                     
                                    v1.1.15                                    
(CVXPY) Oct 17 01:27:10 PM: Your problem has 30 variables, 4 constraints, and 0 parameters.
(CVXPY) Oct 17 01:27:10 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Oct 17 01:27:10 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Oct 17 01:27:10 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Oct 17 01:27:10 PM: Compiling problem (target solver=MOSEK).
(CVXPY) Oct 17 01:27:10 PM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffing -

## Ex 2 : Minimum fuel consumption

In [9]:
# Problem constants
A = np.array([[-1,.4,.8],[1,0,0],[0,1,0]])
B = np.array([[1],[0],[.5]])
x_des = np.array([[7],[2],[-6]])
N = 10
X0 = np.array([0,0,0])
# Variables
u = cp.Variable(N)
x = cp.Variable((3,N+1))
z = cp.Variable(N) # Epigraphical variable
# Objective and constraints
constraints = []
constraints.extend([x[:,0] == X0 , x[:,N] == x_des[:,0]])

for i in range(N):
    constraints.append(x[:,i+1] == A@x[:,i] + B[:,0]*u[i])
    
# Abs. value
# constraints.extend([-z <= u , u <= z , z >= cp.square(u)])
# for i in range(N):
    constraints.append(-z[i]<=u[i])
    constraints.append(u[i]<=z[i])
    constraints.append(z[i] >= cp.square(u[i]))
    
objective = cp.Minimize(cp.sum(z))

# Solve problem
prob = cp.Problem(objective, constraints)
prob.solve(solver=cp.MOSEK, verbose=True)

                                     CVXPY                                     
                                    v1.1.15                                    
(CVXPY) Oct 23 08:31:21 AM: Your problem has 53 variables, 42 constraints, and 0 parameters.
(CVXPY) Oct 23 08:31:21 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Oct 23 08:31:21 AM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Oct 23 08:31:21 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Oct 23 08:31:21 AM: Compiling problem (target solver=MOSEK).
(CVXPY) Oct 23 08:31:21 AM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffing 

70.48736551790535

## Ex 3 : Minimum time path problem 

In [12]:
import cvxpy as cp
# Problem parameters
x0 = [0,0]
xf = [4,2.5]
v1, eta2, eta3 = 1, 1.5, 1.3
v2, v3 = v1/eta2, v1/eta3
# Variables
p = cp.Variable(2)
l = cp.Variable(3)
# Constraints and objective
constraints = []
objective = cp.Minimize(l[0]/v1 + l[1]/v2 + l[2]/v3) # Minimize total travel time
constraints.append(l[0] >= cp.norm(cp.vstack([p[0],1])))
constraints.append(l[1] >= cp.norm(cp.vstack([p[1]-p[0],1])))
constraints.append(l[2] >= cp.norm(cp.vstack([4-p[1],.5])))
# Solve problem
prob = cp.Problem(objective, constraints)
prob.solve(solver=cp.MOSEK, verbose=True)

                                     CVXPY                                     
                                    v1.1.15                                    
(CVXPY) Oct 23 09:51:05 AM: Your problem has 5 variables, 3 constraints, and 0 parameters.
(CVXPY) Oct 23 09:51:05 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Oct 23 09:51:05 AM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Oct 23 09:51:05 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Oct 23 09:51:05 AM: Compiling problem (target solver=MOSEK).
(CVXPY) Oct 23 09:51:05 AM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffing ->

5.719230520394078